In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
%matplotlib inline
import os
import pickle
import re

### Load Data

In [5]:
# Define path to weather data
weather_data_path = '/home/cneiderer/Metis/boston_weather_data'
# Load data
weather_df = pd.DataFrame()
for f in os.listdir(weather_data_path):
    fparts = os.path.splitext(f)
    if fparts[1] == '.csv':
        weather_df = pd.concat([weather_df, pd.read_csv(os.path.join(weather_data_path, f))], axis=0)

/home/cneiderer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,11,14,15,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,67,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,22,23,25,67,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/cneiderer/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,11,84,86) have mixed types. Specify dtype option on impo

### Select Data

In [9]:
def select_and_rename_cols(df):
    
    # Select hourly columns only
    cols = ['DATE']
    for c in df.columns[7:]:
        if c[:6].lower() == 'hourly':
            cols.append(c)
    df = df[cols]

    # Remove hourly from column names
    cols = []
    for c in df.columns:
        cols.append(re.sub('hourly', '', c.lower()))
    df.columns = cols  
    
    # Select relevant columns
    df = df[['date', 'skyconditions', 'visibility', 'prsentweathertype', 
             'drybulbtempf', 'drybulbtempc', 'relativehumidity', 
             'windspeed', 'winddirection', 'stationpressure', 
             'pressuretendency', 'pressurechange', 'precip']]
    
    # Rename columns
    df.columns = ['MeasurementTime', 'SkyConditions', 'Visibility', 'WeatherType', 
                  'TempC', 'TempF', 'Humidity', 'WindSpeed', 'WindDirection',
                  'Pressure', 'PressureTendency', 'PressureChange', 'Precip']
    
    return df

In [13]:
weather_df = select_and_rename_cols(weather_df)
weather_df.head()

,MeasurementTime,SkyConditions,Visibility,WeatherType,TempC,TempF,Humidity,WindSpeed,WindDirection,Pressure,PressureTendency,PressureChange,Precip
0,2018-01-01 00:54,CLR:00,10.00,NaN,2,-16.7,59.0,16,320,30.17,1.0,NaN,0.00
1,2018-01-01 01:00,NaN,NaN,NaN,2,-16.7,59.0,16,320,30.18,1.0,-0.01,NaN
2,2018-01-01 01:54,CLR:00,10.00,NaN,1,-17.2,59.0,11,300,30.19,NaN,NaN,0.00
3,2018-01-01 02:54,CLR:00,10.00,NaN,1,-17.2,54.0,21,290,30.19,NaN,NaN,0.00
4,2018-01-01 03:54,CLR:00,10.00,NaN,1,-17.2,54.0,18,290,30.18,0.0,NaN,0.00


In [35]:
# tmp = weather_df[weather_df.MeasurementTime.apply(lambda x: x[-2:]=='54')]

In [ ]:
# Save weather df
with open('raw_weather.pkl', 'wb') as f:
    pickle.dump(weather_df, f)

### Map Sky Codes

In [ ]:
# NOAA AU weather code mapping
sky_map = {'': '', 
           'CLR': 'Clear',
           'FEW': 'Few',
           'SCT': 'Scattered',
           'BKN': 'Broken',
           'OVC': 'Overcast',
           'VV': 'Obscured'}

def map_sky_codes(code):

    # Check for nans (which are type float)
    if isinstance(code, float):
        return ''
    
    # Map obscured levels to VV for consitency
    sky = re.sub('(09|10)(?=:)', 'VV', code)
    # Remove all digits 
    sky = re.sub('\d+', ' ', sky)
    # Replace all colons
    sky = re.sub(':', '', sky)
    # Strip leading/trailing whitespace and split into list
    sky = sky.strip().split()
    
    # Extract description from sky map
    if 'OVC' in sky:
        return sky_map['OVC']
    elif 'BKN' in sky:
        return sky_map['BKN']
    elif 'SCT' in sky:
        return sky_map['SCT']
    elif 'FEW' in sky:
        return sky_map['FEW']
    elif 'CLR' in sky:
        return sky_map['CLR']
    elif 'VV' in sky:
        return sky_map['VV']
    else:
        return ''

In [ ]:
weather_df['SkyText'] = weather_df.SkyConditions.apply(map_sky_codes)

### Map Weather Codes

In [211]:
# # NOAA AU weather code mapping
# au_map = {'': '', 
#           'DZ:01': 'Drizzle',
#           'RA:02': 'Rain',
#           'SN:03': 'Snow',
#           'SG:04': 'Snow Grains',
#           'IC:05': 'Ice Crystals',
#           'PL:06': 'Ice Pellets',
#           'GR:07': 'Hail',
#           'GS:08': 'Small Hail and/or Snow Pellets',
#           'UP:09': 'Unknown Precipitation',
#           'BR:1': 'Mist',
#           'FG:2': 'Fog',
#           'FU:3': 'Smoke',
#           'VA:4': 'Volcanic Ash',
#           'DU:5': 'Widespread Dust',
#           'SA:6': 'Sand',
#           'HZ:7': 'Haze',
#           'PY:8': 'Spray',
#           'PO:1': 'Well developed dust/sand whirls',
#           'SQ:2': 'Squalls',
#           'FC:3': 'Funnel CLoud, Waterspout or Tornado',
#           'SS:4': 'Sandstorm',
#           'DS:5': 'Duststorm'}

In [212]:
# NOAA AU weather code prefix mapping
au_pre = {'': '',
          '-': 'Light',
          '+': 'Heavy',
          'VC': 'vicinity (apparent but not at point of observation)',
          'MI': 'shallow',
          'PR': 'partial',
          'BC': 'patches',
          'DR': 'low drifting',
          'BL': 'blowing',
          'SH': 'showers',
          'TS': 'thunderstorm',
          'FZ': 'freezing'}

In [213]:
# NOAA AU weather code mapping
au_map = {'': '', 
          'DZ': 'Drizzle',
          'RA': 'Rain',
          'SN': 'Snow',
          'SG': 'Snow Grains',
          'IC': 'Ice Crystals',
          'PL': 'Ice Pellets',
          'GR': 'Hail',
          'GS': 'Small Hail and/or Snow Pellets',
          'UP': 'Unknown Precipitation',
          'BR': 'Mist',
          'FG': 'Fog',
          'FU': 'Smoke',
          'VA': 'Volcanic Ash',
          'DU': 'Widespread Dust',
          'SA': 'Sand',
          'HZ': 'Haze',
          'PY': 'Spray',
          'PO': 'Well developed dust/sand whirls',
          'SQ': 'Squalls',
          'FC': 'Funnel CLoud, Waterspout or Tornado',
          'SS': 'Sandstorm',
          'DS': 'Duststorm',
          '-': 'Light',
          '+': 'Heavy',
          'VC': 'vicinity',
          'MI': 'shallow',
          'PR': 'partial',
          'BC': 'patches',
          'DR': 'low drifting',
          'BL': 'blowing',
          'SH': 'showers',
          'TS': 'thunderstorm',
          'FZ': 'freezing'}

In [214]:
# NOAA AW weather code mapping
aw_map = {'': '',
          'HZ:04': 'Haze, smoke, or dust in suspension in the air, visibility equal to or greater than 1km',
          'FU:05': 'Smoke',
          'DU:07': 'Dust or sand raised by wind at or near the station at the time of observation, ' +
                   'but no well‐developed dust whirl(s) or sand, whirl(s), and no duststorm or sandstorm seen or, ' + 
                   'in the case of ships, blowing spray at the station',
          'BR:10': 'Mist',
          '11': 'Diamond dust',
          '12': 'Distant lightning',
          'SQ:18': 'Squalls',
          '20': 'Fog (during preceding hour but not at time of observation)',
          '21': 'Precipitation (during preceding hour but not at time of observation)',
          '22': 'Drizzle (not freezing) or snow grains (during preceding hour but not at time of observation)',
          '23': 'Rain (not freezing) (during preceding hour but not at time of observation)',
          '24': 'Snow (during preceding hour but not at time of observation)',
          '25': 'Freezing drizzle or freezing rain (during preceding hour but not at time of observation)',
          '26': 'Thunderstorm (with or without precipitation) (during preceding hour but not at time of observation)',
          '27': 'Blowing or drifting snow or sand (during preceding hour but not at time of observation)',
          '28': 'Blowing or drifting snow or sand, visibility equal to or greater than 1 km ' + 
                '(during preceding hour but not at time of observation)',
          '29': 'Blowing or drifting snow or sand, visibility less than 1 km ' + 
                '(during preceding hour but not at time of observation)',
          'FG:30': 'Fog',
          'FG:31': 'Fog or ice fog in patches',
          'FG:32': 'Fog or ice fog, has become thinner during the past hour',
          'FG:33': 'Fog or ice fog, no appreciable change during the past hour',
          'FG:34': 'Fog or ice fog, has begun or become thicker during the past hour',
          'FG:35': 'Fog, depositing rime',
          '40': 'Precipitation',
          '41': 'Precipitation, slight or moderate',
          '42': 'Precipitation, heavy',
          '43': 'Liquid precipitation, slight or moderate',
          '44': 'Liquid precipitation, heavy',
          '45': 'Solid precipitation, slight or moderate',
          '46': 'Solid precipitation, heavy',
          '47': 'Freezing precipitation, slight or moderate',
          '48': 'Freezing precipitation, heavy',
          'DZ:50': 'Drizzle',
          'DZ:51': 'Drizzle, slight',
          'DZ:52': 'Drizzle, moderate',
          'DZ:53': 'Drizzle, heavy',
          'FZDZ:54': 'Drizzle, freezing, slight',
          'FZDZ:55': 'Drizzle, freezing, moderate',
          'FZDZ:56': 'Drizzle, freezing, heavy',
          'DZ:57': 'Drizzle and rain, slight',
          'DZ:58': 'Drizzle and rain, moderate or heavy',
          'RA:60': 'Rain',
          'RA:61': 'Rain, slight',
          'RA:62': 'Rain, moderate',
          'RA:63': 'Rain, heavy',
          'FZRA:64': 'Rain, freezing, slight',
          'FZRA:65': 'Rain, freezing, moderate',
          'FZRA:66': 'Rain, freezing, heavy',
          'RA:67': 'Rain or drizzle and snow, slight',
          'RA:68': 'Rain or drizzle and snow, moderate or heavy',
          'SN:70': 'Snow',
          'SN:71': 'Snow, slight',
          'SN:72': 'Snow, moderate',
          'SN:73': 'Snow, heavy',
          'PL:74': 'Ice pellets, slight',
          'PL:75': 'Ice pellets, moderate',
          'PL:76': 'Ice pellets, heavy',
          'SG:77': 'Snow grains',
          'IC:78': 'Ice crystals',
          '80': 'Showers or intermittent precipitation',
          'SHRA:81': 'Rain showers or intermittent rain, slight',
          'SHRA:82': 'Rain showers or intermittent rain, moderate',
          'SHRA:83': 'Rain showers or intermittent rain, heavy',
          'SHRA:84': 'Rain showers or intermittent rain, violent',
          'SHSN:85': 'Snow showers or intermittent snow, slight',
          'SHSN:86': 'Snow showers or intermittent snow, moderate',
          'SHSN:87': 'Snow showers or intermittent snow, heavy',
          'HAIL:89': 'Hail',
          'TS': 'Thunderstorm',
          'TS:90': 'Thunderstorm',
          'TS:91': 'Thunderstorm, slight or moderate, with no precipitation',
          'TS:92': 'Thunderstorm, slight or moderate, with rain showers and/or snow showers',
          'TS:93': 'Thunderstorm, slight or moderate, with hail',
          'HAIL:93': 'Thunderstorm, slight or moderate, with hail',
          'TS:94': 'Thunderstorm, heavy, with no precipitation',
          'TS:95': 'Thunderstorm, heavy, with rain showers and/or snow',
          'TS+HAIL:96': 'Thunderstorm, heavy, with hail',
          '+FC:99': 'Tornado'}

In [227]:
# NOAA MW weather code mapping
mw_map = {'FU:04': 'Visibility reduced by smoke, e.g. veldt or forest fires, industrial smoke or volcanic ashes',
          'HZ:05': 'Haze',
          'DU:06': 'Widespread dust in suspension in the air, not raised by wind at or near the station at the ' +
                   'time of observation',
          'DU:07': 'Dust or sand raised by wind at or near the station at the time of observation, ' +
                   'but no well‐developed dust whirl(s) or sand whirl(s), and no duststorm or sandstorm seen or, ' + 
                   'in the case of ships, blowing spray at the station',
          'DU:08': 'Well developed dust whirl(s) or sand whirl(s) seen at or near the station ' +
                   'during the preceding hour or at the time of observation, but no duststorm or sandstorm',
          'DU:09': 'Duststorm or sandstorm within sight at the time of observation, or at the station ' +
                   'during the preceding hour',
          'BR:10': 'Mist',
          'FG:11': 'Patches of shallow fog or ice fog at the station, whether on land or sea, ' +
                   'not deeper than about 2 meters on land or 10 meters at sea',
          'FG:12': 'More or less continuous shallow fog or ice fog at the station, whether on land or sea, ' + 
                   'not deeper than about 2 meters on land or 10 meters at sea',
          '13': 'Lightning visible, no thunder heard',
          '14': 'Precipitation within sight, not reaching the ground or the surface of the sea',
          '15': 'Precipitation within sight, reaching the ground or the surface of the sea, but distant, ' +
                'i.e., estimated to be more than 5 km from the station',
          '16': 'Precipitation within sight, reaching the ground or the surface of the sea, near to, but not at the station',
          'TS:17': 'Thunderstorm, but no precipitation at the time of observation',
          'SQ:18': 'Squalls at or within sight of the station during the preceding hour or at the time of observation',
          'FC:19': 'Funnel cloud(s) (Tornado cloud or waterspout) at or within sight of the station ' +
                   'during the preceding hour or at the time of observation',
          '20': 'Drizzle (not freezing) or snow grains not falling as shower(s) ' +
                '(during the preceding hour but not at the time of observation)',
          '21': 'Rain (not freezing) not falling as shower(s) ' +
                '(during the preceding hour but not at the time of observation)',
          '22': 'Snow not falling as shower(s) ' +
                '(during the preceding hour but not at the time of observation)',
          '23': 'Rain and snow or ice pellets not falling as shower(s) ' + 
                '(during the preceding hour but not at the time of observation)',
          '24': 'Freezing drizzle or freezing rain not falling as shower(s) ' +
                '(during the preceding hour but not at the time of observation)',
          '25': 'Shower(s) of rain ' +
                '(during the preceding hour but not at the time of observation)',
          '26': 'Shower(s) of snow or of rain and snow ' +
                '(during the preceding hour but not at the time of observation)',
          '27': 'Shower(s) of hail (Hail, small hail, snow pellets), or rain and hail ' +
                '(during the preceding hour but not at the time of observation)',
          '28': 'Fog or ice fog ' +
                '(during the preceding hour but not at the time of observation)',
          '29': 'Thunderstorm (with or without precipitation) ' + 
                '(during the preceding hour but not at the time of observation)',
          'DU:30': 'Slight or moderate duststorm or sandstorm has decreased during the preceding hour',
          'DU:31': 'Slight or moderate duststorm or sandstorm no appreciable change during the preceding hour',
          'DU:32': 'Slight or moderate duststorm or sandstorm has begun or has increased during the preceding hour',
          'DU:33': 'Severe duststorm or sandstorm has decreased during the preceding hour',
          'DU:34': 'Severe duststorm or sandstorm no appreciable change during the preceding hour',
          'DU:35': 'Severe duststorm or sandstorm has begun or has increased during the preceding hour',
          'DRSN:36': 'Slight or moderate drifting snow generally low (below eye level)',
          'DRSN:37': 'Heavy drifting snow generally low (below eye level)',
          'BLSN:38': 'Slight or moderate blowing snow generally high (above eye level)',
          'BLSN:39': 'Heavy blowing snow generally high (above eye level)',
          'FG:40': 'Fog or ice fog at a distance at the time of observation, ' +
                   'but not at the station during the preceding hour, the fog or ice fog extending to a level above that of the observer',
          'FG:41': 'Fog or ice fog in patches',
          'FG:42': 'Fog or ice fog, sky visible, has become thinner during the preceding hour',
          'FG:43': 'Fog or ice fog, sky invisible, has become thinner during the preceding hour',
          'FG:44': 'Fog or ice fog, sky visible, no appreciable change during the preceding hour',
          'FG:45': 'Fog or ice fog, sky invisible, no appreciable change during the preceding hour',
          'FG:46': 'Fog or ice fog, sky visible, has begun or has become thicker during the preceding hour',
          'FG:47': 'Fog or ice fog, sky invisible, has begun or has become thicker during the preceding hour',
          'FG:48': 'Fog, depositing rime, sky visible',
          'FG:49': 'Fog, depositing rime, sky invisible',
          'DZ:50': 'Drizzle, not freezing, intermittent, slight at time of observation',
          'DZ:51': 'Drizzle, not freezing, continuous, slight at time of observation',
          'DZ:52': 'Drizzle, not freezing, intermittent, moderate at time of observation',
          'DZ:53': 'Drizzle, not freezing, continuous, moderate at time of observation',
          'DZ:54': 'Drizzle, not freezing, intermittent, heavy (dense) at time of observation',
          'DZ:55': 'Drizzle, not freezing, continuous, heavy (dense) at time of observation',
          'FZDZ:56': 'Drizzle, freezing, slight',
          'FZDZ:57': 'Drizzle, freezing, moderate or heavy (dense)',
          'DZ:58': 'Drizzle and rain, slight',
          'DZ:59': 'Drizzle and rain, moderate or heavy',
          'RA:60': 'Rain, not freezing, intermittent, slight at time of observation',
          'RA:61': 'Rain, not freezing, continuous, slight at time of observation',
          'RA:62': 'Rain, not freezing, intermittent, moderate at time of observation',
          'RA:63': 'Rain, not freezing, continuous, moderate at time of observation',
          'RA:64': 'Rain, not freezing, intermittent, heavy at time of observation',
          'RA:65': 'Rain, not freezing, continuous, heavy at time of observation',
          'FZRA:66': 'Rain, freezing, slight',
          'FZRA:67': 'Rain, freezing, moderate or heavy',
          'RA:68': 'Rain or drizzle and snow, slight',
          'RA:69': 'Rain or drizzle and snow, moderate or heavy',
          'SN:70': 'Intermittent fall of snowflakes, slight at time of observation',
          'SN:71': 'Continuous fall of snowflakes, slight at time of observation',
          'SN:72': 'Intermittent fall of snowflakes, moderate at time of observation',
          'SN:73': 'Continuous fall of snowflakes, moderate at time of observation',
          'SN:74': 'Intermittent fall of snowflakes, heavy at time of observation',
          'SN:75': 'Continuous fall of snowflakes, heavy at time of observation',
          '76': 'Diamond dust (with or without fog)',
          'SG:77': 'Snow grains (with or without fog)',
          'SN:78': 'Isolated star-like snow crystals (with or without fog)',
          'PL:79': 'Ice pellets',
          'SHRA:80': 'Rain shower(s), slight',
          'SHRA:81': 'Rain shower(s), moderate or heavy',
          'SHRA:82': 'Rain shower(s), violent',
          'SHRASN:83': 'Shower(s) of rain and snow mixed, slight',
          'SHRASN:84': 'Shower(s) of rain and snow mixed, moderate or heavy',
          'SHSN:85': 'Show shower(s), slight',
          'SHSN:86': 'Snow shower(s), moderate or heavy',
          'SH:87': 'Shower(s) of snow pellets or small hail, with or without rain or rain and snow mixed, slight',
          'SH:88': 'Shower(s) of snow pellets or small hail, with or without rain or rain and snow mixed, moderate or heavy',
          'SH:89': 'Shower(s) of hail (hail, small hail, snow pellets), with or without rain or rain and snow mixed, ' +
                   'not associated with thunder, slight',
          'SH:90': 'Shower(s) of hail (hail, small hail, snow pellets), with or without rain or rain and snow mixed, ' +
                   'not associated with thunder moderate or heavy',
          'RA:91': 'Slight rain at time of observation, thunderstorm during the preceding hour but not at time of observation',
          'RA:92': 'Moderate or heavy rain at time of observation, thunderstorm during the preceding hour but not at time of observation',
          '93': 'Slight snow, or rain and snow mixed or hail (Hail, small hail, snow pellets), at time of observation, ' +
                'thunderstorm during the preceding hour but not at time of observation',
          '94': 'Moderate or heavy snow, or rain and snow mixed or hail(Hail, small hail, snow pellets) at time of observation, ' +
                'thunderstorm during the preceding hour but not at time of observation',
          'TS:95': 'Thunderstorm, slight or moderate, without hail (Hail, small hail, snow pellets), ' +
                   'but with rain and/or snow at time of observation, thunderstorm at time of observation',
          'TS:96': 'Thunderstorm, slight or moderate, with hail (hail, small hail, snow pellets) at time of observation, ' +
                   'thunderstorm at time of observation',
          'TS:97': 'Thunderstorm, heavy, without hail (Hail, small hail, snow pellets), but with rain and/or snow at time of observation,' +
                   'thunderstorm at time of observation',
          'TS:98': 'Thunderstorm combined with duststorm or sandstorm at time of observation, thunderstorm at time of observation',
          'TS:99': 'Thunderstorm, heavy, with hail (Hail, small hail, snow pellets) at time of observation, thunderstorm at time of observation'}

In [228]:
mw_map['RA:65']

'Rain, not freezing, continuous, heavy at time of observation'

In [223]:
def map_au_code(code):    
    pattern = '(\w{2})*(\+|\-)*(\w+\:*\d*)'
    search = re.compile(pattern)
    grps = search.findall(code)[0]
    weather_str = ' '.join([au_map[grps[1]], au_map[grps[2]], au_map[grps[0]]]).strip()    
    return weather_str

In [224]:
def map_aw_code(code):    
    pattern = '(\w+\:*\d*)'
    search = re.compile(pattern)
    grps = search.findall(code)[0]
    weather_str = ' '.join([aw_map[grps]]).strip()
    return weather_str

In [229]:
def map_mw_code(code):
    pattern = '(\w+\:*\d*)'
    search = re.compile(pattern)
    grps = search.findall(code)[0]
    weather_str = ' '.join([mw_map[grps]]).strip()
    return weather_str

In [230]:
test = set(weather_df.WeatherType)

In [231]:
test

{'* |* |',
 '+DZ:01 FG:2 |FG:30 s DZ:53 s |',
 '+GR:07 RA:02 |RA:60 s TS:90 s |SH:90 s',
 '+PL:06 RA:02 BR:1 |RA:63 PL:76 |',
 '+RA:02 BR:1 |RA:63 TS:90 |',
 '+RA:02 BR:1 |RA:63 TS:90 |SHRA:82 TS:97',
 '+RA:02 BR:1 |RA:63 TS:90 |TS:95',
 '+RA:02 BR:1 |RA:63 |',
 '+RA:02 BR:1 |RA:63 |RA:61',
 '+RA:02 BR:1 |RA:63 |RA:65',
 '+RA:02 BR:1 |SQ:18 RA:63 |',
 '+RA:02 BR:1 |s RA:63 s TS:90 s |',
 '+RA:02 BR:1 |s RA:63 s |',
 '+RA:02 FG:2 |FG:30 RA:63 TS:90 |',
 '+RA:02 FG:2 |FG:30 RA:63 TS:90 |FG:44 SHRA:82 TS:97',
 '+RA:02 FG:2 |FG:30 RA:63 |',
 '+RA:02 FG:2 |FG:30 RA:63 |FG:44 RA:65',
 '+RA:02 s BR:1 s |s RA:63 s |',
 '+RA:02 |RA:63 TS:90 TS:95 |',
 '+RA:02 |RA:63 TS:90 |',
 '+RA:02 |RA:63 TS:90 |SHRA:82 TS:97',
 '+RA:02 |RA:63 s TS:90 s TS:95 |',
 '+RA:02 |RA:63 s TS:90 s |SHRA:82 TS:97',
 '+RA:02 |RA:63 |',
 '+RA:02 |RA:63 |RA:65',
 '+SN:03 FG:2 |FG:30 SN:73 |',
 '+SN:03 FG:2 |FG:30 SN:73 |FG:44',
 '+SN:03 FG:2 |FG:33 FG:35 s SN:73 s |',
 '+SN:03 FG:2 |FG:35 SN:73 |',
 '+SN:03 FG:2 |FG:35 S

In [234]:
w = []
for ii, code in enumerate(test):

    # Check for nans (which are type float)
    if isinstance(code, float):
        w.append(['', '', ''])
        print(code, ['', '', ''])
        continue
    
    # Remove random floating s- and *-characters
    code = re.sub('(s|\*)\s*', '', code)
    # Split on pipes
    code = code.split('|')
    # Strip leading/trailing whitespace
    code = [x.strip() for x in code]
    
    weather = []
    for idx, c in enumerate(code):
        if len(c) == 0:
            weather.append('')
            continue
        
        # Remove colon and digits for AU codes and prefixes since they don't matter in AU codes
        if idx == 0:            
            c = re.sub(':*\d+', '', c)            
            
        # Split code into parts
        cparts = c.split()
        # Compile weather string from code parts
        weather_str = []
        for jdx, c2 in enumerate(cparts):
            if idx == 0:
                weather_str.append(map_au_code(c2))
            elif idx == 1:
                weather_str.append(map_aw_code(c2))
            else:
                weather_str.append(map_mw_code(c2))
        # Compile weather observations
        weather.append(' '.join(weather_str).strip())
        
    # Compile all weather data
    w.append(weather)
    print(code, weather)

nan ['', '', '']
['-RA:02', 'RA:61', ''] ['Light Rain', 'Rain, slight', '']
['', '', 'RA:65'] ['', '', 'Rain, not freezing, continuous, heavy at time of observation']
['RA:02', 'RA:62 TS:90 TS:92', ''] ['Rain', 'Rain, moderate Thunderstorm Thunderstorm, slight or moderate, with rain showers and/or snow showers', '']
['', 'RA:61', ''] ['', 'Rain, slight', '']
['+RA:02', 'RA:63 TS:90 TS:95', ''] ['Heavy Rain', 'Rain, heavy Thunderstorm Thunderstorm, heavy, with rain showers and/or snow', '']
['', '', 'DZ:51'] ['', '', 'Drizzle, not freezing, continuous, slight at time of observation']
['RA:02', 'TS:91', ''] ['Rain', 'Thunderstorm, slight or moderate, with no precipitation', '']
['RA:02', 'RA:62', 'RA:63'] ['Rain', 'Rain, moderate', 'Rain, not freezing, continuous, moderate at time of observation']
['-RA:02 SN:03 BR:1', 'RA:61 RA:67', ''] ['Light Rain Snow Mist', 'Rain, slight Rain or drizzle and snow, slight', '']
['-SN:03 SN:03', 'SN:71', ''] ['Light Snow Snow', 'Snow, slight', '']
['FU

In [235]:
w

[['', '', ''],
 ['Light Rain', 'Rain, slight', ''],
 ['', '', 'Rain, not freezing, continuous, heavy at time of observation'],
 ['Rain',
  'Rain, moderate Thunderstorm Thunderstorm, slight or moderate, with rain showers and/or snow showers',
  ''],
 ['', 'Rain, slight', ''],
 ['Heavy Rain',
  'Rain, heavy Thunderstorm Thunderstorm, heavy, with rain showers and/or snow',
  ''],
 ['', '', 'Drizzle, not freezing, continuous, slight at time of observation'],
 ['Rain', 'Thunderstorm, slight or moderate, with no precipitation', ''],
 ['Rain',
  'Rain, moderate',
  'Rain, not freezing, continuous, moderate at time of observation'],
 ['Light Rain Snow Mist', 'Rain, slight Rain or drizzle and snow, slight', ''],
 ['Light Snow Snow', 'Snow, slight', ''],
 ['Smoke',
  'Haze, smoke, or dust in suspension in the air, visibility equal to or greater than 1km Smoke',
  ''],
 ['', '', 'Thunderstorm, but no precipitation at the time of observation'],
 ['Fog',
  'Fog',
  'Fog or ice fog, sky invisible, n

#### Legacy Code

In [ ]:
#
def replace_nans(x):
    if isinstance(x, float):
        if np.isnan(x):
            return ''
        else:
            return str(x)        
    return re.sub('s', '', x)

In [ ]:
#
def map_weather_codes(code):

    if code == '':
        return([''])
            
    split1 = code.split('|')
    weather = []
    weather_grps = []
    for idx in range(len(wsplit)):
        split2 = split1[idx].split()
        for jdx in range(len(split2)):
            weather_str = ''                        
            if idx == 0:
                pattern = '(VC|MI|PR|BC|DR|BL|SH|TS|FZ)*(\+|-)*(\w+:\d+)*'
                search = re.compile(pattern)
                grps = search.findall(split2[jdx])[0]
                
                if (len(au_pre[grps[1]]) > 0) & (len(au_map[grps[2]]) > 0):
                    join_str = ' '
                else:
                    join_str = ''
                    
                weather_str = au_pre[grps[1]] + join_str + au_map[grps[2]]
            elif idx == 1:
                pattern = '(\w+:\d+)*'
                search = re.compile(pattern)
                grps = search.findall(split2[jdx])
                weather_str = aw_map[grps[0]]
            elif idx == 2:
                pass
            weather_grps.append(weather_str)
        weather.append(weather_grps)
        
#     print(weather)
    return weather

In [ ]:
weather_df.WeatherType = weather_df.WeatherType.apply(replace_nans)

In [ ]:
weather_df['WeatherText'] = weather_df.WeatherType.apply(map_weather_codes)

In [ ]:
weather_df.head()

In [ ]:
[x[0] for x in weather_df.WeatherText if len(x[0]) > 0]

#### Testing Logic

In [ ]:
weather_description = []
for idx in range(weather_df.WeatherType.shape[0]):
    weather_description.append(weather_df.WeatherType.iloc[idx])

In [ ]:
# Map weather codes to descriptions
weather = []
for w in test: 
    if w == '':
        weather.append('')
        continue
    split1 = w.split('|')
    weather_grps = []
    for idx in range(len(wsplit)):
        split2 = split1[idx].split()
        for jdx in range(len(split2)):
            weather_str = ''                        
            if idx == 0:
                pattern = '(VC|MI|PR|BC|DR|BL|SH|TS|FZ)*(\+|-)*(\w+:\d+)*'
                search = re.compile(pattern)
                grps = search.findall(split2[jdx])[0]
                
                if (len(au_pre[grps[1]]) > 0) & (len(au_map[grps[2]]) > 0):
                    join_str = ' '
                else:
                    join_str = ''
                    
                weather_str = au_pre[grps[1]] + join_str + au_map[grps[2]]
            elif idx == 1:
                pattern = '(\w+:\d+)*'
                search = re.compile(pattern)
                grps = search.findall(split2[jdx])
                weather_str = aw_map[grps[0]]
            elif idx == 2:
                pass
            weather_grps.append(weather_str)
        weather.append(weather_grps)

In [ ]:
weather_df.head(50)

In [ ]:
weather_df.info()

In [ ]:
weather_df.MeasurementTime.iloc[0]

In [ ]:
test = []
for row in x:
#     sky = re.sub(':', '', re.sub('\d+', ' ', re.sub('(09|10)(?=:)', 'VV', row))).strip().split()

    # Map obscured levels to VV for consitency
    sky = re.sub('(09|10)(?=:)', 'VV', row)
    # Remove all digits 
    sky = re.sub('\d+', ' ', sky)
    # Replace all colons
    sky = re.sub(':', '', sky)
    # Strip leading/trailing whitespace and split into list
    sky = sky.strip().split()
    
    # Extract description from sky map
    if 'OVC' in sky:
        test.append(sky_map['OVC'])
    elif 'BKN' in sky:
        test.append(sky_map['BKN'])
    elif 'SCT' in sky:
        test.append(sky_map['SCT'])
    elif 'FEW' in sky:
        test.append(sky_map['FEW'])
    elif 'CLR' in sky:
        test.append(sky_map['CLR'])
    elif 'VV' in sky:
        test.append(sky_map['VV'])
    else:
        test.append('Unknown')

In [ ]:
set(test)

In [ ]:
(sum([testi == 'Unknown' for testi in test]) + sum([testi == 'Obscured' for testi in test])) / len(test)

In [ ]:
x = list(set(weather_df.SkyConditions))[1:]

In [ ]:
x[1570]

In [ ]:
idx_list = [ii for ii, y in enumerate(test) if len(y) == 0]

In [ ]:
[x[idx] for idx in idx_list]

In [ ]:
test[1569:1572]

In [ ]:
#'(\w{3}):\d+\s*\d+\s*(\w{3}):\d+\s*\d+\s*(\w{3}):\d+\s*\d+\s*'

In [ ]:
[idx for idx, x in enumerate(test) if x == 'x']

In [ ]:
test = []
for row in x:
    test.append(re.sub(':', '', re.sub('\d+', ' ', row)).strip().split())

In [ ]:
test

In [ ]:
t = '10:04 8 BKN:07 220'

In [ ]:
re.sub(':', '', re.sub('\d+', ' ', re.sub('(09|10)(?=:)', 'XX', t))).strip().split()

In [ ]:
import itertools
print(list(set(itertools.chain.from_iterable(test))))